1. In this final lab, we will model our data. Import sklearn `train_test_split` and separate the data.

2. Separate X_train and X_test into numerical and categorical (X_train_cat , X_train_num , X_test_cat , X_test_num)

3. Use X_train_num to fit scalers.  Transform BOTH X_train_num and X_test_num.

4. Encode the categorical variables X_train_cat and X_test_cat (See the hint below for encoding categorical data!!!)

5. Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.

6. Try a simple linear regression with all the data to see whether we are getting good results.

7. Great! Now define a function that takes a list of models and train (and tests) them so we can try a lot of them without repeating code.

8. Use the function to check `LinearRegressor` and `KNeighborsRegressor`.

9. You can check also the `MLPRegressor` for this task!

10. Check and discuss the results.  



You should deal with the categorical variables as shown below (for ordinal encoding, dummy code has been provided as well):
Encoder Type | Column 
-----------------|-----------------
One hot | state
Ordinal | coverage
Ordinal | employmentstatus
Ordinal | location code
One hot | marital status
One hot | policy type
One hot | policy
One hot | renew offercustomer_df
One hot | sales channel
One hot | vehicle class
Ordinal | vehicle size
Ordinal | education
One hot | response
One hot | gender


### Dummy code

data["coverage"] = data["coverage"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})

In [369]:
import pandas as pd #tablas https://pandas.pydata.org/docs/user_guide/pyarrow.html
import numpy as np #estadistica https://numpy.org/doc/1.25/user/index.html#user
import matplotlib.pyplot as plt #gráficos https://matplotlib.org/stable/plot_types/index.html
import seaborn as sns #gráficos https://seaborn.pydata.org/tutorial/regression.html
from sklearn import linear_model #modelo de regresion https://scikit-learn.org/stable/
from sklearn.preprocessing import MinMaxScaler # do not use the function Normalise() - it does something entirely different
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from datetime import datetime #https://docs.python.org/3/library/datetime.html
from datetime import timedelta #this is a good time to learn about dates and timestamps. You can find some info here:
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import ttest_1samp
import scipy.stats as st
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.plotting import plot_decision_regions

1. In this final lab, we will model our data. Import sklearn `train_test_split` and separate the data.

In [370]:
df_cat = pd.read_csv('categorical22_08.csv')
df_num = pd.read_csv('numerical.csv')

In [371]:
df_num

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,6979.535903,0,94,13,42,0,8,1131.464935
1,12887.431650,48767,108,18,38,0,2,566.472247
2,7645.861827,0,106,18,65,0,7,529.881344
3,2813.692575,43836,73,12,44,0,1,138.130879
4,8256.297800,62902,69,14,94,0,2,159.383042
...,...,...,...,...,...,...,...,...
8187,4100.398533,47761,104,16,58,0,1,541.282007
8188,23405.987980,71941,73,18,89,0,2,198.234764
8189,3096.511217,21604,79,14,28,0,1,379.200000
8190,8163.890428,0,85,9,37,3,2,790.784983


In [372]:
df_cat

,state,response,coverage,education,effective_to_date,employment_status,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size,Month,Day
0,Arizona,No,Extended,Bachelor,2011-01-31,Unemployed,F,Suburban,Single,Personal Auto,L3,3,Agent,Four-Door Car,Medsize,January,Monday
1,Nevada,No,Premium,Bachelor,2011-02-19,Employed,F,Suburban,Married,Personal Auto,L3,1,Agent,Two-Door Car,Medsize,February,Saturday
2,California,No,Basic,Bachelor,2011-01-20,Unemployed,M,Suburban,Married,Corporate Auto,L2,1,Call Center,SUV,Medsize,January,Thursday
3,Washington,No,Basic,Bachelor,2011-02-03,Employed,M,Rural,Single,Personal Auto,L1,1,Agent,Four-Door Car,Medsize,February,Thursday
4,Oregon,Yes,Basic,Bachelor,2011-01-25,Employed,F,Rural,Married,Personal Auto,L3,2,Web,Two-Door Car,Medsize,January,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8187,California,No,Premium,College,2011-01-06,Employed,F,Suburban,Single,Personal Auto,L2,1,Branch,Four-Door Car,Large,January,Thursday
8188,California,No,Basic,Bachelor,2011-02-10,Employed,M,Urban,Married,Personal Auto,L1,2,Web,Four-Door Car,Medsize,February,Thursday
8189,California,Yes,Extended,College,2011-02-12,Employed,F,Suburban,Divorced,Corporate Auto,L3,1,Branch,Four-Door Car,Medsize,February,Saturday
8190,California,No,Extended,Bachelor,2011-02-06,Unemployed,M,Suburban,Single,Corporate Auto,L2,1,Branch,Four-Door Car,Medsize,February,Sunday


In [373]:
df_cat = df_cat.astype(str)
df_cat

,state,response,coverage,education,effective_to_date,employment_status,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size,Month,Day
0,Arizona,No,Extended,Bachelor,2011-01-31,Unemployed,F,Suburban,Single,Personal Auto,L3,3,Agent,Four-Door Car,Medsize,January,Monday
1,Nevada,No,Premium,Bachelor,2011-02-19,Employed,F,Suburban,Married,Personal Auto,L3,1,Agent,Two-Door Car,Medsize,February,Saturday
2,California,No,Basic,Bachelor,2011-01-20,Unemployed,M,Suburban,Married,Corporate Auto,L2,1,Call Center,SUV,Medsize,January,Thursday
3,Washington,No,Basic,Bachelor,2011-02-03,Employed,M,Rural,Single,Personal Auto,L1,1,Agent,Four-Door Car,Medsize,February,Thursday
4,Oregon,Yes,Basic,Bachelor,2011-01-25,Employed,F,Rural,Married,Personal Auto,L3,2,Web,Two-Door Car,Medsize,January,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8187,California,No,Premium,College,2011-01-06,Employed,F,Suburban,Single,Personal Auto,L2,1,Branch,Four-Door Car,Large,January,Thursday
8188,California,No,Basic,Bachelor,2011-02-10,Employed,M,Urban,Married,Personal Auto,L1,2,Web,Four-Door Car,Medsize,February,Thursday
8189,California,Yes,Extended,College,2011-02-12,Employed,F,Suburban,Divorced,Corporate Auto,L3,1,Branch,Four-Door Car,Medsize,February,Saturday
8190,California,No,Extended,Bachelor,2011-02-06,Unemployed,M,Suburban,Single,Corporate Auto,L2,1,Branch,Four-Door Car,Medsize,February,Sunday


In [374]:
df= pd.concat([df_num, df_cat], axis=1)
df = df.drop(columns=["effective_to_date"])
df


,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount,state,response,...,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size,Month,Day
0,6979.535903,0,94,13,42,0,8,1131.464935,Arizona,No,...,Suburban,Single,Personal Auto,L3,3,Agent,Four-Door Car,Medsize,January,Monday
1,12887.431650,48767,108,18,38,0,2,566.472247,Nevada,No,...,Suburban,Married,Personal Auto,L3,1,Agent,Two-Door Car,Medsize,February,Saturday
2,7645.861827,0,106,18,65,0,7,529.881344,California,No,...,Suburban,Married,Corporate Auto,L2,1,Call Center,SUV,Medsize,January,Thursday
3,2813.692575,43836,73,12,44,0,1,138.130879,Washington,No,...,Rural,Single,Personal Auto,L1,1,Agent,Four-Door Car,Medsize,February,Thursday
4,8256.297800,62902,69,14,94,0,2,159.383042,Oregon,Yes,...,Rural,Married,Personal Auto,L3,2,Web,Two-Door Car,Medsize,January,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8187,4100.398533,47761,104,16,58,0,1,541.282007,California,No,...,Suburban,Single,Personal Auto,L2,1,Branch,Four-Door Car,Large,January,Thursday
8188,23405.987980,71941,73,18,89,0,2,198.234764,California,No,...,Urban,Married,Personal Auto,L1,2,Web,Four-Door Car,Medsize,February,Thursday
8189,3096.511217,21604,79,14,28,0,1,379.200000,California,Yes,...,Suburban,Divorced,Corporate Auto,L3,1,Branch,Four-Door Car,Medsize,February,Saturday
8190,8163.890428,0,85,9,37,3,2,790.784983,California,No,...,Suburban,Single,Corporate Auto,L2,1,Branch,Four-Door Car,Medsize,February,Sunday


In [375]:
df.to_csv("23_08_df.csv", index=False) 

2. Separate X_train and X_test into numerical and categorical (X_train_cat , X_train_num , X_test_cat , X_test_num)

In [376]:
X = df.drop(['total_claim_amount'],axis=1)
y = df['total_claim_amount']

In [377]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

3. Use X_train_num to fit scalers.  Transform BOTH X_train_num and X_test_num.

In [378]:
X_train_num = X_train.select_dtypes(include = np.number)
X_test_num  = X_test.select_dtypes(include = np.number)
X_train_cat = X_train.select_dtypes(include = object)
X_test_cat  = X_test.select_dtypes(include = object)

In [379]:
X_train_num

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
6638,4739.346630,49447,119,9,48,2,1
2747,32306.285870,36301,118,11,85,0,2
1662,3788.649817,48887,95,26,25,0,1
1749,5181.620895,97131,64,5,80,0,6
2336,4529.112791,0,62,22,57,0,9
...,...,...,...,...,...,...,...
3927,4334.064096,23904,123,1,67,3,1
8039,5100.088201,0,71,9,2,1,7
5955,3728.873486,83632,94,7,86,4,1
6936,8390.122077,0,78,22,3,0,2


In [380]:
X_test_cat

,state,response,coverage,education,employment_status,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size,Month,Day
6350,Oregon,No,Basic,High School or Below,Employed,F,Suburban,Married,Personal Auto,L3,2,Agent,SUV,Medsize,February,Friday
8178,California,Yes,Basic,Master or above,Employed,M,Suburban,Married,Corporate Auto,L2,1,Branch,Four-Door Car,Small,January,Thursday
7398,Arizona,No,Premium,College,Employed,M,Rural,Married,Personal Auto,L3,2,Agent,Four-Door Car,Medsize,February,Friday
1288,Oregon,Yes,Basic,High School or Below,Retired,M,Suburban,Divorced,Personal Auto,L3,1,Branch,SUV,Medsize,February,Friday
5710,Washington,No,Basic,High School or Below,Employed,F,Suburban,Married,Personal Auto,L3,2,Branch,Four-Door Car,Small,February,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5647,Oregon,No,Basic,Master or above,Employed,F,Suburban,Single,Personal Auto,L2,2,Agent,Two-Door Car,Small,February,Monday
5940,Nevada,No,Basic,High School or Below,Medical Leave,F,Urban,Single,Personal Auto,L3,3,Call Center,Sports Car,Medsize,January,Wednesday
4246,Arizona,No,Basic,High School or Below,Unemployed,F,Suburban,Married,Personal Auto,L3,4,Agent,Four-Door Car,Medsize,February,Wednesday
7169,Nevada,Yes,Basic,Bachelor,Disabled,M,Suburban,Married,Personal Auto,L2,1,Branch,Sports Car,Large,February,Friday


In [381]:
MinMaxtransformer = MinMaxScaler().fit(X_train_num) #IMPORTANTE ESTO ES EL NOMBRE DEL TRANSFORMADOR
transformer = MinMaxScaler().fit(X_train_num) #Aplicar el min y el max, pero recuerda que X  tiene que ser valores numericos
x_train_norm= transformer.transform(X_train_num) #Convertirlo entre 0 y 1 
print(x_train_norm.shape)
x_train_norm
x_train_normalized= pd.DataFrame(x_train_norm, columns=X_train_num.columns) #df.normalizado para asi poder tener un data set normalizado al que llamar
x_train_normalized

(6553, 7)


,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
0,0.039283,0.494564,0.245763,0.290323,0.510638,0.4,0.000
1,0.420407,0.363079,0.241525,0.354839,0.904255,0.0,0.125
2,0.026139,0.488963,0.144068,0.838710,0.265957,0.0,0.000
3,0.045397,0.971495,0.012712,0.161290,0.851064,0.0,0.625
4,0.036376,0.000000,0.004237,0.709677,0.606383,0.0,1.000
...,...,...,...,...,...,...,...
6548,0.033680,0.239085,0.262712,0.032258,0.712766,0.6,0.000
6549,0.044270,0.000000,0.042373,0.290323,0.021277,0.2,0.750
6550,0.025312,0.836479,0.139831,0.225806,0.914894,0.8,0.000
6551,0.089756,0.000000,0.072034,0.709677,0.031915,0.0,0.125


In [382]:
x_test_norm= transformer.transform(X_test_num) #Convertirlo entre 0 y 1 
print(x_test_norm.shape)
x_test_norm
x_test_normalized= pd.DataFrame(x_test_norm, columns=X_test_num.columns) #df.normalizado para asi poder tener un data set normalizado al que llamar
x_test_normalized

(1639, 7)


,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
0,0.093761,0.558526,0.203390,0.129032,0.276596,0.0,0.250
1,0.049516,0.560156,0.029661,1.000000,0.319149,0.2,0.250
2,0.094789,0.240166,0.216102,0.967742,0.968085,0.2,0.250
3,0.117668,0.202318,0.169492,0.096774,0.457447,0.2,0.125
4,0.012735,0.880837,0.038136,0.612903,0.627660,0.0,0.000
...,...,...,...,...,...,...,...
1634,0.009607,0.357118,0.016949,0.612903,0.478723,0.0,0.000
1635,0.365412,0.208030,0.211864,0.645161,0.297872,0.0,0.125
1636,0.009077,0.000000,0.038136,0.612903,0.287234,0.0,0.000
1637,0.028430,0.183785,0.177966,0.258065,0.531915,0.0,0.000


4. Encode the categorical variables X_train_cat and X_test_cat (See the hint below for encoding categorical data!!!)

One encoding everything :)

In [383]:
X_train_cat = X_train.select_dtypes(include = object)
X_test_cat  = X_test.select_dtypes(include = object)

In [384]:

encoder = OneHotEncoder(handle_unknown='error', drop='first').fit(X_train_cat )
categoricals_train_encoded = encoder.transform(X_train_cat).toarray()
categoricals_test_encoded = encoder.transform(X_test_cat ).toarray()

In [385]:
categorical_train_encoded_df = pd.DataFrame(categoricals_train_encoded)
categorical_train_encoded_df

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6548,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6549,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
6550,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6551,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [386]:
categorical_test_encoded_df = pd.DataFrame(categoricals_test_encoded)
categorical_test_encoded_df

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1634,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1635,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1636,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1637,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


5. Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.

In [387]:
X_train_norm_enc = pd.concat([x_train_normalized, categorical_train_encoded_df], axis=1)
X_train_norm_enc

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,0,1,2,...,32,33,34,35,36,37,38,39,40,41
0,0.039283,0.494564,0.245763,0.290323,0.510638,0.4,0.000,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.420407,0.363079,0.241525,0.354839,0.904255,0.0,0.125,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.026139,0.488963,0.144068,0.838710,0.265957,0.0,0.000,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.045397,0.971495,0.012712,0.161290,0.851064,0.0,0.625,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.036376,0.000000,0.004237,0.709677,0.606383,0.0,1.000,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6548,0.033680,0.239085,0.262712,0.032258,0.712766,0.6,0.000,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6549,0.044270,0.000000,0.042373,0.290323,0.021277,0.2,0.750,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
6550,0.025312,0.836479,0.139831,0.225806,0.914894,0.8,0.000,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6551,0.089756,0.000000,0.072034,0.709677,0.031915,0.0,0.125,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [388]:
X_train_norm_enc.columns = X_train_norm_enc.columns.astype(str)

In [389]:
X_test_norm_enc = pd.concat([x_test_normalized, categorical_test_encoded_df], axis=1)
X_test_norm_enc

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,0,1,2,...,32,33,34,35,36,37,38,39,40,41
0,0.093761,0.558526,0.203390,0.129032,0.276596,0.0,0.250,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.049516,0.560156,0.029661,1.000000,0.319149,0.2,0.250,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.094789,0.240166,0.216102,0.967742,0.968085,0.2,0.250,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.117668,0.202318,0.169492,0.096774,0.457447,0.2,0.125,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.012735,0.880837,0.038136,0.612903,0.627660,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1634,0.009607,0.357118,0.016949,0.612903,0.478723,0.0,0.000,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1635,0.365412,0.208030,0.211864,0.645161,0.297872,0.0,0.125,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1636,0.009077,0.000000,0.038136,0.612903,0.287234,0.0,0.000,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1637,0.028430,0.183785,0.177966,0.258065,0.531915,0.0,0.000,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [390]:
X_test_norm_enc.columns = X_test_norm_enc.columns.astype(str)

In [391]:
X_train_norm_enc.dtypes

customer_lifetime_value          float64
income                           float64
monthly_premium_auto             float64
months_since_last_claim          float64
months_since_policy_inception    float64
number_of_open_complaints        float64
number_of_policies               float64
0                                float64
1                                float64
2                                float64
3                                float64
4                                float64
5                                float64
6                                float64
7                                float64
8                                float64
9                                float64
10                               float64
11                               float64
12                               float64
13                               float64
14                               float64
15                               float64
16                               float64
17              

In [392]:
X_test_norm_enc.dtypes

customer_lifetime_value          float64
income                           float64
monthly_premium_auto             float64
months_since_last_claim          float64
months_since_policy_inception    float64
number_of_open_complaints        float64
number_of_policies               float64
0                                float64
1                                float64
2                                float64
3                                float64
4                                float64
5                                float64
6                                float64
7                                float64
8                                float64
9                                float64
10                               float64
11                               float64
12                               float64
13                               float64
14                               float64
15                               float64
16                               float64
17              

In [393]:
y

0       1131.464935
1        566.472247
2        529.881344
3        138.130879
4        159.383042
           ...     
8187     541.282007
8188     198.234764
8189     379.200000
8190     790.784983
8191     369.600000
Name: total_claim_amount, Length: 8192, dtype: float64

6. Try a simple linear regression with all the data to see whether we are getting good results.

In [394]:
lm = linear_model.LinearRegression()
lm.fit(X_train_norm_enc,y_train)

LinearRegression()

In [395]:
predictions = lm.predict(X_test_norm_enc)
r2_score(y_test, predictions)

0.7628538500501031

7. Great! Now define a function that takes a list of models and train (and tests) them so we can try a lot of them without repeating code.

In [396]:
def train_and_test_regression_models(models, X_train, X_test, y_train, y_test, test_size=0.2, random_state=100):
    results = {}

    for model in models:
        model_name = model.__class__.__name__
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        results[model_name] = r2

    return results

8. Use the function to check `LinearRegressor` and `KNeighborsRegressor`.

In [397]:

X_train = X_train_norm_enc
X_test= X_test_norm_enc

models_to_try = [LinearRegression(), KNeighborsRegressor()]

results = train_and_test_regression_models(models_to_try, X_train, X_test, y_train, y_test)
print(results)


{'LinearRegression': 0.7628538500501031, 'KNeighborsRegressor': 0.5496134807500319}



 9. You can check also the `MLPRegressor` for this task!

In [398]:
from sklearn.neural_network import MLPRegressor
models_to_try = [MLPRegressor()]

results = train_and_test_regression_models(models_to_try, X_train, X_test, y_train, y_test)
print(results)

{'MLPRegressor': 0.779855666888571}


/Users/agustinredondoarena/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


10. All of them have a similar explanatory power, besides the KNN (not K defined)